# Books Recommender System Uisng Clustering | Collaborative Based

In [1]:
# importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)

In [2]:
# reading the books dataset
books = pd.read_csv("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\raw_data\\BX-Books.csv", sep = ";", on_bad_lines= "skip", encoding = "latin-1")

In [3]:
# first two records of the books dataset
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [4]:
# shape of the books dataset
books.shape

(271360, 8)

In [5]:
# feature names
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [6]:
# now let's select only important features from the books dataset
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [7]:
# let's see the first two records
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [8]:
# let's rename the column names such that it is easy for us to call the feature names.
books.rename(columns = {
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "url"
                        },
             inplace=True)

In [9]:
# now let's see the first two records with new column names
books.head(2)

,ISBN,title,author,year,publisher,url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [10]:
# now let's read the users dataset
users = pd.read_csv("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\raw_data\\BX-Users.csv", sep = ";", on_bad_lines= "skip", encoding = "latin-1")

In [11]:
# first 5 records of the users data
users.head(5)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
# now let's check the shape of the users dataframe
users.shape

(278858, 3)

In [13]:
# now let's read the ratings dataframe
ratings = pd.read_csv("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\raw_data\\BX-Book-Ratings.csv", sep = ";", on_bad_lines= "skip", encoding = "latin-1")

In [14]:
# now let's check the first ten records
ratings.head(10)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [15]:
# let's change the name of the columns
ratings.rename(columns =
               {
    "User-ID": "user_id",
    "Book-Rating": "rating"
               },
        inplace =True)

In [16]:
# let's check the name of the new rating columns
ratings.columns

Index(['user_id', 'ISBN', 'rating'], dtype='object')

In [17]:
# let's see the shape of the three data frame
books.shape, users.shape, ratings.shape

((271360, 6), (278858, 3), (1149780, 3))

In [18]:
ratings["user_id"].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [19]:
# let's capture the index values of all users who rated more than 200 books
ind =(ratings["user_id"].value_counts()>200)
ind = ind[ind].index
ind

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [20]:
# only 899 unique customers have rated morethan 200 books
len(ind)

899

In [21]:
# now let's filter all the records in the ratings dataframe with the name rating
ratings= ratings[ratings["user_id"].isin(ind)]

In [22]:
# now let's see the shape of this dataframe
ratings.shape

(526356, 3)

In [23]:
# visualizing the new ratings dataframe
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [24]:
# now let's merge the ratings and books dataframe on the common feature name which is ISBN
# here we perform an inner join.
ratings_with_books =ratings.merge(books, on="ISBN")

In [25]:
# let's see the shape of the new dataframe
ratings_with_books.shape

(487671, 8)

In [26]:
# let's visualize the new dataframe
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [27]:
# now let's perform a group by operation to see which book got frequently rated
num_rating = ratings_with_books.groupby("title")["rating"].count().reset_index()

In [28]:
# visualizing the num_rating data
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [29]:
num_rating.rename(columns={
    "rating":"num_of_rating"
},
                  inplace=True)

In [30]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [31]:
final_rating = ratings_with_books.merge(num_rating, on="title")

In [32]:
# 
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7


In [33]:
final_rating.shape

(487671, 9)

In [34]:
# now let's filter out those books which get morethan 50 ratings
final_rating = final_rating[final_rating["num_of_rating"]>=50]

In [35]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [36]:
# now let's see sample records
final_rating.sample(4)

,user_id,ISBN,rating,title,author,year,publisher,url,num_of_rating
187507,107784,0804108749,5,Saint Maybe,ANNE TYLER,1992,Ivy Books,http://images.amazon.com/images/P/0804108749.0...,73
464809,261829,0385722206,0,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE,2002,Anchor,http://images.amazon.com/images/P/0385722206.0...,111
5413,2276,0446607657,0,When the Wind Blows,James Patterson,1999,Warner Vision,http://images.amazon.com/images/P/0446607657.0...,175
419694,236283,0515132187,0,The Villa,Nora Roberts,2002,Jove Books,http://images.amazon.com/images/P/0515132187.0...,97


In [37]:
# let's see the shape of our final dataframe
final_rating.shape

(61853, 9)

In [38]:
# now let's get rid of some duplicated informations
final_rating.drop_duplicates(["title","user_id"], inplace=True)

In [39]:
# shape of the final dataframe
final_rating.shape

(59850, 9)

In [40]:
final_rating.columns

Index(['user_id', 'ISBN', 'rating', 'title', 'author', 'year', 'publisher',
       'url', 'num_of_rating'],
      dtype='object')

In [41]:
# now let's create the pivot table
book_pivot = final_rating.pivot_table(columns = "user_id", index = "title", values = "rating")

In [42]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,6323,6543,6563,6575,7158,7286,7346,7915,8067,8245,8681,8936,9856,10447,10819,11601,11676,11993,12538,12824,12982,13082,13273,13552,13850,14422,14521,15408,15418,15957,16106,16634,16795,16916,16966,17950,18067,18401,19085,19664,20115,20201,21014,21252,21576,21659,22625,23288,23768,23872,23902,24194,24921,25409,25601,25981,26516,26535,26544,26583,26593,27617,27647,28204,28360,28523,28591,28634,29259,29855,30276,30487,30509,30511,30533,30711,30735,30972,31315,31391,31556,31826,31846,32195,32440,32721,32773,33974,35050,35433,35836,35857,35859,36606,36609,36836,36907,37567,37712,37950,38023,38273,38556,38781,39281,39467,39616,39646,39773,40889,40943,41084,41841,42400,42914,43246,43806,43842,44296,44595,44728,46398,47316,48494,49109,49460,49889,50225,50547,50730,51094,51350,51386,51450,51883,52199,52203,52584,52614,52853,52917,53174,53628,53729,54218,55187,55490,55492,55548,55734,55892,56271,56360,56399,56447,56856,56959,59172,59727,59971,60244,60277,60337,60707,61619,62272,62891,62895,63394,63714,63938,64436,65258,66680,67840,68555,69042,69078,69232,69355,69405,69697,69808,69971,70052,70415,70594,71712,72352,72992,73394,73651,73681,75591,75860,76151,76223,76352,76499,76626,76818,76942,77809,77940,78553,78783,78834,78973,79186,79441,80538,80683,81045,81492,81560,81977,82407,82511,82831,82893,82926,83287,83637,84024,85426,85526,85656,85701,85757,85993,86202,86243,86641,87141,87143,87555,87746,87974,88283,88677,88693,88733,88793,88937,89192,89602,91203,91342,91832,91931,92547,92652,92810,92853,92979,93047,93085,93363,93629,94242,94347,94853,94923,94985,95010,95193,95316,95359,95903,95932,95991,96054,96448,97874,98297,98391,98741,98758,100459,100644,100846,100906,101209,101305,101851,101876,102275,102359,102647,102702,102967,104399,104429,104636,104665,105374,105517,105979,106225,106816,107021,107301,107453,107784,107951,108005,108285,108352,109461,109574,109901,109955,110029,110214,110483,110912,110934,110973,111174,111947,112001,112026,112881,113270,113519,113817,113904,113983,114178,114368,114414,114444,114865,114868,114988,115002,115003,115120,115490,116122,116599,117251,117539,119575,120093,120548,121726,122429,122793,122881,123625,123790,123883,123981,124078,124079,124363,124487,124942,125039,125203,125519,125692,125774,126492,126604,126736,127200,127233,127359,127429,127914,128208,128696,128835,129008,129074,129358,129465,129716,129851,130474,130554,130571,131027,131046,131402,131675,131837,131855,132083,133567,133571,133689,133747,133868,134837,135045,135149,135265,135458,135831,136010,136139,136205,136252,136382,137190,137589,137688,138073,138097,138441,138578,138844,139467,139742,140000,140036,140358,141493,141710,141819,141901,141902,142093,142454,142524,143175,143253,143415,143792,144131,144531,144555,145165,145449,145451,145619,145641,146113,146230,146348,147141,147451,147847,147965,148199,148258,148744,149069,149907,149908,149934,150124,150498,150968,150979,151589,151790,151806,152186,152249,153563,153621,153662,153718,154176,154992,155014,155147,155219,155495,155916,156150,156269,156467,157247,157273,157811,158295,159033,159376,159858,160032,160541,160819,161752,162052,162639,162738,163804,163973,164027,164096,164323,164465,164533,164675,164828,165232,165308,165319,166123,166596,167349,167471,167800,168047,168064,168144,168245,168387,168464,169233,169663,169699,170184,170229,170513,170518,170575,170634,170742,170947,171118,172030,172512,172742,172888,173291,173415,173632,173684,173835,174092,174216,174304,174791,175003,175886,177072,177090,177374,177432,177458,177690,178181,178199,178667,178950,179733,179744,179772,179978,180348,180586,180651,180917,180957,181176,181687,182085,182086,182987,182993,183196,183287,183995,184299,185233,185308,185384,186570,187145,187170,187256,187517,187520,188010,188951,189334,189516,189835,189973,190459,190708,190807,190925,191187,192093,193499,193560,194600,194669,195694,196047,196077,196160,196457,196502,197364,197659,197775,198621

In [43]:
book_pivot.shape

(742, 888)

In [44]:
# now let's fill the NaN values with the 0 rating
book_pivot.fillna(0, inplace = True)

In [45]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,6323,6543,6563,6575,7158,7286,7346,7915,8067,8245,8681,8936,9856,10447,10819,11601,11676,11993,12538,12824,12982,13082,13273,13552,13850,14422,14521,15408,15418,15957,16106,16634,16795,16916,16966,17950,18067,18401,19085,19664,20115,20201,21014,21252,21576,21659,22625,23288,23768,23872,23902,24194,24921,25409,25601,25981,26516,26535,26544,26583,26593,27617,27647,28204,28360,28523,28591,28634,29259,29855,30276,30487,30509,30511,30533,30711,30735,30972,31315,31391,31556,31826,31846,32195,32440,32721,32773,33974,35050,35433,35836,35857,35859,36606,36609,36836,36907,37567,37712,37950,38023,38273,38556,38781,39281,39467,39616,39646,39773,40889,40943,41084,41841,42400,42914,43246,43806,43842,44296,44595,44728,46398,47316,48494,49109,49460,49889,50225,50547,50730,51094,51350,51386,51450,51883,52199,52203,52584,52614,52853,52917,53174,53628,53729,54218,55187,55490,55492,55548,55734,55892,56271,56360,56399,56447,56856,56959,59172,59727,59971,60244,60277,60337,60707,61619,62272,62891,62895,63394,63714,63938,64436,65258,66680,67840,68555,69042,69078,69232,69355,69405,69697,69808,69971,70052,70415,70594,71712,72352,72992,73394,73651,73681,75591,75860,76151,76223,76352,76499,76626,76818,76942,77809,77940,78553,78783,78834,78973,79186,79441,80538,80683,81045,81492,81560,81977,82407,82511,82831,82893,82926,83287,83637,84024,85426,85526,85656,85701,85757,85993,86202,86243,86641,87141,87143,87555,87746,87974,88283,88677,88693,88733,88793,88937,89192,89602,91203,91342,91832,91931,92547,92652,92810,92853,92979,93047,93085,93363,93629,94242,94347,94853,94923,94985,95010,95193,95316,95359,95903,95932,95991,96054,96448,97874,98297,98391,98741,98758,100459,100644,100846,100906,101209,101305,101851,101876,102275,102359,102647,102702,102967,104399,104429,104636,104665,105374,105517,105979,106225,106816,107021,107301,107453,107784,107951,108005,108285,108352,109461,109574,109901,109955,110029,110214,110483,110912,110934,110973,111174,111947,112001,112026,112881,113270,113519,113817,113904,113983,114178,114368,114414,114444,114865,114868,114988,115002,115003,115120,115490,116122,116599,117251,117539,119575,120093,120548,121726,122429,122793,122881,123625,123790,123883,123981,124078,124079,124363,124487,124942,125039,125203,125519,125692,125774,126492,126604,126736,127200,127233,127359,127429,127914,128208,128696,128835,129008,129074,129358,129465,129716,129851,130474,130554,130571,131027,131046,131402,131675,131837,131855,132083,133567,133571,133689,133747,133868,134837,135045,135149,135265,135458,135831,136010,136139,136205,136252,136382,137190,137589,137688,138073,138097,138441,138578,138844,139467,139742,140000,140036,140358,141493,141710,141819,141901,141902,142093,142454,142524,143175,143253,143415,143792,144131,144531,144555,145165,145449,145451,145619,145641,146113,146230,146348,147141,147451,147847,147965,148199,148258,148744,149069,149907,149908,149934,150124,150498,150968,150979,151589,151790,151806,152186,152249,153563,153621,153662,153718,154176,154992,155014,155147,155219,155495,155916,156150,156269,156467,157247,157273,157811,158295,159033,159376,159858,160032,160541,160819,161752,162052,162639,162738,163804,163973,164027,164096,164323,164465,164533,164675,164828,165232,165308,165319,166123,166596,167349,167471,167800,168047,168064,168144,168245,168387,168464,169233,169663,169699,170184,170229,170513,170518,170575,170634,170742,170947,171118,172030,172512,172742,172888,173291,173415,173632,173684,173835,174092,174216,174304,174791,175003,175886,177072,177090,177374,177432,177458,177690,178181,178199,178667,178950,179733,179744,179772,179978,180348,180586,180651,180917,180957,181176,181687,182085,182086,182987,182993,183196,183287,183995,184299,185233,185308,185384,186570,187145,187170,187256,187517,187520,188010,188951,189334,189516,189835,189973,190459,190708,190807,190925,191187,192093,193499,193560,194600,194669,195694,196047,196077,196160,196457,196502,197364,197659,197775,198621

In [49]:
# we need to consider only the ratings greater than 0 in our model development,
# we have to ignore the 0 values,
from scipy.sparse import csr_matrix
book_sparce = csr_matrix(book_pivot)

In [51]:
book_sparce

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14961 stored elements in Compressed Sparse Row format>

In [52]:
# lets import our clustering algorithm
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = "brute")

In [53]:
model

NearestNeighbors(algorithm='brute')

In [54]:
model.fit(book_sparce)

NearestNeighbors(algorithm='brute')

In [58]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors = 6)

In [60]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [61]:
suggestion

array([[237, 238, 240, 241, 184, 536]], dtype=int64)

In [63]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [64]:
book_pivot.index[3]

'4 Blondes'

In [65]:
books_name = book_pivot.index

In [66]:
books_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [67]:
import pickle

In [74]:
# let's save the processed data
pickle.dump(final_rating, open("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\processed_data\\final_rating.pkl","wb"))
pickle.dump(book_pivot, open("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\processed_data\\book_pivot.pkl","wb"))
pickle.dump(books_name, open("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\data\\processed_data\\books_name.pkl","wb"))

In [76]:
# now let's save the models
pickle.dump(model,open("C:\\Users\\yozil\\Desktop\\My_Projects\\Book_Recommendation\\models\\model.pkl", "wb"))

In [77]:
# now let's test the recommendation system using functions
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)
    

In [78]:
book_name = "A Bend in the Road"
recommend_book(book_name)

A Bend in the Road
Exclusive
The Cradle Will Fall
No Safe Place
Family Album
Last Man Standing


In [80]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
